# Unconstrained global optimization with Scipy

**TODO**:
* Plots:
    1. Les solutions visitées dans l'espace des solutions (2D avec pcolormesh et contour)
    2. f(x*) w.t. iteration number
    3. error w.t. iteration number (=> + fit to have a clear view of the convergence rate ?)
    4. error w.t. number of function evaluations + error w.t. *total* number of function evaluations (i.e. including the number of gradient and hessian evaluations)
    5. error w.t. execution time
    6. (benchmark session ! distinguish the derivative-free to the non-derivative free case) average version of 3., 4., 5. over several runs with random initial state (+ error bar or box plot)
    7. (benchmark session) err w.t. algorithms parameters (plot the iteration or evaluation number or execution time to reach in average an error lower than N% with e.g. N=99%)

## Plot functions

In [ ]:
%matplotlib inline

import numpy as np

import matplotlib
matplotlib.rcParams['figure.figsize'] = (8, 8)

import math

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

import time

In [ ]:
a = np.array([[1, 2, 3],[4, 5, 6]])
b = np.array([[-1, 2, 3],[-4, 5, 6]])
np.amin(np.hstack([a, b]), axis=1)


In [ ]:
def plot_2d_contour_solution_space(func, xmin=-np.ones(2), xmax=np.ones(2), xstar=None, xvisited=None, title=""):
    fig, ax = plt.subplots(figsize=(12, 8))
    
    if xvisited is not None:
        xmin = np.amin(np.hstack([xmin.reshape([-1, 1]), xvisited]), axis=1)
        xmax = np.amax(np.hstack([xmax.reshape([-1, 1]), xvisited]), axis=1)

    x1_space = np.linspace(xmin[0], xmax[0], 200)
    x2_space = np.linspace(xmin[1], xmax[1], 200)
    
    x1_mesh, x2_mesh = np.meshgrid(x1_space, x2_space)

    zz = func(np.array([x1_mesh.ravel(), x2_mesh.ravel()])).reshape(x1_mesh.shape)
    
    ############################
    
    min_value = func(xstar)
    max_value = zz.max()
    
    levels = np.logspace(0.1, 3., 5)          # TODO

    im = ax.pcolormesh(x1_mesh, x2_mesh, zz,
                       vmin=0.1,              # TODO
                       vmax=max_value,
                       norm=colors.LogNorm(), # TODO
                       shading='gouraud',
                       cmap='gnuplot2') # 'jet' # 'gnuplot2'

    plt.colorbar(im, ax=ax)

    cs = plt.contour(x1_mesh, x2_mesh, zz, levels,
                     linewidths=(2, 2, 2, 2, 3),
                     linestyles=('dotted', '-.', 'dashed', 'solid', 'solid'),
                     alpha=0.5,
                     colors='white')
    ax.clabel(cs, inline=False, fontsize=12)

    ############################

    if xvisited is not None:
        ax.plot(xvisited[0],
                xvisited[1],
                '-og',
                alpha=0.5,
                label="$visited$")

    ############################

    if xstar is not None:
        sc = ax.scatter(xstar[0],
                   xstar[1],
                   c='red',
                   label="$x^*$")
        sc.set_zorder(10)        # put this point above every thing else

    ############################

    ax.set_title(title)

    ax.set_xlabel(r"$x_1$")
    ax.set_ylabel(r"$x_2$")

    ax.legend(fontsize=12)

    plt.show()

In [ ]:
def plot_2d_solution_space(func, xmin=-np.ones(2), xmax=np.ones(2), xstar=None, xvisited=None, angle_view=None, title=""):
    fig = plt.figure(figsize=(12, 8))
    ax = axes3d.Axes3D(fig)
    
    if angle_view is not None:
        ax.view_init(angle_view[0], angle_view[1])

    x1_space = np.linspace(xmin[0], xmax[0], 100)
    x2_space = np.linspace(xmin[1], xmax[1], 100)
    
    x1_mesh, x2_mesh = np.meshgrid(x1_space, x2_space)

    zz = func(np.array([x1_mesh.ravel(), x2_mesh.ravel()])).reshape(x1_mesh.shape)   # TODO

    ############################
        
    surf = ax.plot_surface(x1_mesh,
                           x2_mesh,
                           zz,
                           cmap='gnuplot2', # 'jet' # 'gnuplot2'
                           norm=colors.LogNorm(),   # TODO
                           rstride=1,
                           cstride=1,
                           #color='b',
                           shade=False)

    ax.set_zlabel(r"$f(x_1, x_2)$")

    fig.colorbar(surf, shrink=0.5, aspect=5)

    ############################

    if xstar is not None:
        ax.scatter(xstar[0],
                   xstar[1],
                   func(xstar),
                   #s=50,          # TODO
                   c='red',
                   alpha=1,
                   label="$x^*$")
    
    ax.set_title(title)
    
    ax.set_xlabel(r"$x_1$")
    ax.set_ylabel(r"$x_2$")

    ax.legend(fontsize=12)
    
    plt.show()

## The "basin-hopping" algorithm

Basin-hopping is a **stochastic** algorithm which attempts to find the **global** minimum of a function.

Official documentation:
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.basinhopping.html#scipy.optimize.basinhopping

In [ ]:
import numpy as np
import scipy
from scipy import optimize
import matplotlib.pyplot as plt

In [ ]:
%%time

NDIM = 2

x_list = []
fx_list = []

def callback(x, f, accept):
    x_list.append(x)
    fx_list.append(f)
    print(len(x_list), x, f, accept)
    
func = scipy.optimize.rosen
x0 = np.random.uniform(-100., 100., size=NDIM)

res = optimize.basinhopping(func,
                            x0,                # The initial point
                            niter=100,         # The number of basin hopping iterations
                            callback=callback,
                            disp=False)        # Print status messages

print("x* =", res.x)
print("f(x*) =", res.fun)
print("Cause of the termination:", ";".join(res.message))
print("Number of evaluations of the objective functions:", res.nfev)
print("Number of evaluations of the jacobian:", res.njev)
print("Number of iterations performed by the optimizer:", res.nit)

In [ ]:
res

In [ ]:
plot_2d_contour_solution_space(scipy.optimize.rosen,
                               xmin=-2*np.ones(2),
                               xmax=2*np.ones(2),
                               xstar=res.x,
                               xvisited=np.array(x_list).T,
                               title="Rosenbrock function")

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, squeeze=True, figsize=(12, 12))

ax = ax.ravel()

# Res w.t. iteration number
ax[0].semilogy(fx_list)
ax[0].set_xlabel("f(x)")
ax[0].set_ylabel("Error")

# Err w.t. iteration number
ax[1].semilogy(np.array([fx_list]).T - actual_fxstar)
ax[1].set_xlabel("Iteration number")
ax[1].set_ylabel("Error")

# Err w.t. time
ax[2].semilogy(time_list, np.array([fx_list]).T - actual_fxstar)
ax[2].set_xlabel("Time (sec)")
ax[2].set_ylabel("Error")

# Time w.t. error
ax[3].semilogx(np.array([fx_list]).T - actual_fxstar, time_list)
ax[3].set_xlabel("Error")
ax[3].set_ylabel("Time (sec)");

## The "Differential Evolution" (DE) algorithm

Differential Evolution is a **stochastic** algorithm which attempts to find the **global** minimum of a function.

Official documentation:
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution

More information:
* [Practical advice](http://www1.icsi.berkeley.edu/~storn/code.html#prac)
* [Wikipedia article](https://en.wikipedia.org/wiki/Differential_evolution)

In [ ]:
%%time

NDIM = 2

func = scipy.optimize.rosen
actual_xstar = np.ones(NDIM)
actual_fxstar = 0.
bounds = [(-10, 10) for d in range(NDIM)]

x_list = []
fx_list = []
time_list = []

def callback(xk, convergence):
    x_list.append(xk)
    fx_list.append(func(xk))
    time_list.append(time.time() - init_time)
    print(len(x_list), xk, fx_list[-1], convergence)

init_time = time.time()

res = optimize.differential_evolution(func,
                                      bounds,              # The initial point
                                      maxiter=100,         # The number of DE iterations
                                      callback=callback,
                                      #polish=False,
                                      disp=False)          # Print status messages

print("x* =", res.x)
print("f(x*) =", res.fun)
print("Cause of the termination:", res.message)
print("Number of evaluations of the objective functions:", res.nfev)
print("Number of iterations performed by the optimizer:", res.nit)

In [ ]:
res

In [ ]:
plot_2d_contour_solution_space(scipy.optimize.rosen,
                               xmin=-2*np.ones(2),
                               xmax=2*np.ones(2),
                               xstar=res.x,
                               xvisited=np.array(x_list).T,
                               title="Rosenbrock function")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, squeeze=True, figsize=(12, 6))

ax = ax.ravel()

## Res w.t. iteration number (useless...)
#ax[0].semilogy(fx_list)
#ax[0].set_xlabel("Iteration number")
#ax[0].set_ylabel("f(x)")

# Err w.t. iteration number
ax[0].semilogy(np.array([fx_list]).T - actual_fxstar)
ax[0].set_xlabel("Iteration number")
ax[0].set_ylabel("Error")

# Err w.t. time
ax[1].semilogy(time_list, np.array([fx_list]).T - actual_fxstar)
ax[1].set_xlabel("Time (sec)")
ax[1].set_ylabel("Error");

## Time w.t. error (useless...)
#ax[3].semilogx(np.array([fx_list]).T - actual_fxstar, time_list)
#ax[3].set_xlabel("Error")
#ax[3].set_ylabel("Time (sec)");

## The "simulated annealing" algorithm

This algorithm has been replaced by the "basin-hopping" algorithm since Scipy 0.15.

See the official documentation for more details: https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.anneal.html.

## Benchmark

### Workaround to get the number of function evaluations

This is not a good solution to get num func eval...
It's much easier and save (at least we have the true value) to count and store that in the objective function instead of trying to retrieve it in solvers (it's fairly easy with DE but not with basinhopping). But don't forget to reinit the counter in the objective function before each minimization... => use pyai's objective functions.

In [ ]:
%%time

NDIM = 2

func = scipy.optimize.rosen
actual_xstar = np.ones(NDIM)
actual_fxstar = 0.
bounds = [(-10, 10) for d in range(NDIM)]

x_list = []
fx_list = []
time_list = []
fev_list = []

def callback(xk, convergence):
    x_list.append(xk)
    fx_list.append(func(xk))
    time_list.append(time.time() - init_time)
    fev_list.append(solver._nfev)
    print(len(x_list), xk, fx_list[-1], fev_list[-1], convergence)

solver = optimize._differentialevolution.DifferentialEvolutionSolver(func,
                                                                     bounds,
                                                                     maxiter=100,
                                                                     callback=callback,
                                                                     #polish=False,
                                                                     disp=False)

init_time = time.time()

res = solver.solve()

print("x* =", res.x)
print("f(x*) =", res.fun)
print("Cause of the termination:", res.message)
print("Number of evaluations of the objective functions:", res.nfev)
print("Number of iterations performed by the optimizer:", res.nit)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, squeeze=True, figsize=(15, 5))

ax = ax.ravel()

## Res w.t. iteration number (useless...)
#ax[0].semilogy(fx_list)
#ax[0].set_xlabel("Iteration number")
#ax[0].set_ylabel("f(x)")

# Err w.t. iteration number
ax[0].semilogy(np.array([fx_list]).T - actual_fxstar)
ax[0].set_xlabel("Iteration number")
ax[0].set_ylabel("Error")

# Err w.t. time
ax[1].semilogy(time_list, np.array([fx_list]).T - actual_fxstar)
ax[1].set_xlabel("Time (sec)")
ax[1].set_ylabel("Error")

# Time w.t. error
ax[2].loglog(fev_list, np.array([fx_list]).T - actual_fxstar)
ax[2].set_xlabel("Num fonction evaluations")
ax[2].set_ylabel("Error")

plt.tight_layout(); # Fix plot margins errors